Prova a automatizzare la selezione della linea sotto tramite l'immagine a colori \
nell'ultimo box c'è la selezione automatizzata

In [63]:
import cv2
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas as pd


Load video

In [64]:
# # Define the relative path to the video file
# notebook_dir = Path().resolve()
# project_root = notebook_dir.parent.parent
# video_path = project_root / "data" / "recording_2" / "Recording_2_normal_speed.mp4"
# video_path = str(video_path)

# # Load the video
# cap = cv2.VideoCapture(video_path)

# # Check
# print(f"Opened: {cap.isOpened()}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, Total Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}")

Define range for mask

In [65]:
# # Define the range for light brown color in HSV
# lower_brown = np.array([00, 30, 100])
# upper_brown = np.array([20, 200, 255])

# # Define the range for rose color in HSV
# lower_rose = np.array([150, 30, 200])
# upper_rose = np.array([180, 200, 255])

Define tolerance for horizontal line detection

In [66]:
# # Tolleranza in radianti (5 gradi)
# tolerance = np.deg2rad(5)

get frame

In [67]:
# # Set frame position
# cap.set(cv2.CAP_PROP_POS_FRAMES, 50)
# ret, image_color = cap.read()
# if not ret:
#     print("Failed to read the frame")
#     cap.release()
#     exit()

Apply mask

In [68]:
# # Convert the image to HSV color space
# hsv_image = cv2.cvtColor(image_color, cv2.COLOR_BGR2HSV)

# # Create masks for brown and rose colors
# mask_brown = cv2.inRange(hsv_image, lower_brown, upper_brown)
# mask_rose = cv2.inRange(hsv_image, lower_rose, upper_rose)

# # Combine the masks
# combined_mask = cv2.bitwise_or(mask_brown, mask_rose)

# # apply brown and rose mask
# extracted_image = cv2.bitwise_and(image_color, image_color, mask=combined_mask)

# # Show the extracted image
# plt.imshow(cv2.cvtColor(extracted_image, cv2.COLOR_BGR2RGB))
# plt.title("Extracted Image")
# plt.axis('off')
# plt.show()

Cut the bottom part

In [69]:
# # Crop the bottom part of the image
# limit_y = math.floor(3/4*extracted_image.shape[0])
# bottom_image = extracted_image[limit_y:extracted_image.shape[0], 0:extracted_image.shape[1]]
# plt.imshow(cv2.cvtColor(bottom_image, cv2.COLOR_BGR2RGB))
# plt.title("Bottom Image")
# plt.axis('off')
# plt.show()

Get edges

In [70]:
# # Convert the bottom image to grayscale
# gray_image = cv2.cvtColor(bottom_image, cv2.COLOR_BGR2GRAY)

# # Compute Otsu's threshold 
# otsu_thresh, _ = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# # Set lower and upper thresholds relative to Otsu's threshold
# lower = 0.5 * otsu_thresh
# upper = 1.5 * otsu_thresh

# # get edges
# edges = cv2.Canny(gray_image, lower, upper)

# # Show the edges
# plt.imshow(edges, cmap='gray')
# plt.title("Edges")
# plt.axis('off')
# plt.show()

Get lines

In [71]:
# # Apply Probabilistic Hough Line Transform (allow to set minLineLength and maxLineGap)
# min_line_length = 50
# max_line_gap = 10 
# lines_p = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=min_line_length, maxLineGap=max_line_gap) 

# # Draw the lines on the image
# image_with_lines = np.copy(bottom_image)
# if lines_p is not None:
#     for line in lines_p:
#         x1, y1, x2, y2 = line[0]
#         cv2.line(image_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)

# # Show the image with lines
# plt.imshow(cv2.cvtColor(image_with_lines, cv2.COLOR_BGR2RGB))
# plt.title("Image with Lines")
# plt.axis('off')
# plt.show()

Convert the lines in the form (rho, theta)

In [72]:
# '''
# Sructure of lines_p:
# (x1, y1, x2, y2) are the coordinates of the two end points of the line segment
# I want to convert them to the form rho,theta in order to easly detect the horizontal lines
# '''
# def convert_to_rho_theta(x1, y1, x2, y2):
#     """
#     Convert a line from (x1, y1, x2, y2) format to (rho, theta).
#     """
#     # Compute the line angle (in radians) and find the normal angle
#     theta = np.arctan2(y2 - y1, x2 - x1) + np.pi / 2  # Normal angle

#     # Compute rho using the normal equation
#     rho = x1 * np.cos(theta) + y1 * np.sin(theta)

#     # Ensure rho is positive by adjusting theta accordingly
#     if rho < 0:
#         rho = -rho
#         theta += np.pi  # Shift by 180 degrees to maintain equivalence

#     return rho, theta

# # Convert all lines to (rho, theta)
# if lines_p is not None:
#     rho_theta_lines = []
#     for line in lines_p:
#         x1, y1, x2, y2 = line[0]  # Extract line coordinates
#         rho, theta = convert_to_rho_theta(x1, y1, x2, y2)
#         rho_theta_lines.append((rho, theta))



Get horizontal lines

In [73]:
# # Filtra le linee orizzontali
# if rho_theta_lines is not None:
#     horizontal_lines = [line for line in rho_theta_lines if abs(line[1] - np.pi/2) <= tolerance]
#     print(f"Number of horizontal lines: {len(horizontal_lines)}")
    
#     # Draw the horizontal lines on the image
#     image_with_horizontal_lines = np.copy(bottom_image)

#     for line in horizontal_lines:
#         rho, theta = line
#         a = np.cos(theta)
#         b = np.sin(theta)
#         x0 = a * rho
#         y0 = b * rho
#         x1 = int(x0 + 1000 * (-b))
#         y1 = int(y0 + 1000 * (a))
#         x2 = int(x0 - 1000 * (-b))
#         y2 = int(y0 - 1000 * (a))
#         cv2.line(image_with_horizontal_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)
        

#     # Show the image with horizontal lines
#     plt.imshow(cv2.cvtColor(image_with_horizontal_lines, cv2.COLOR_BGR2RGB))
#     plt.title("Image with Horizontal Lines")
#     plt.axis('off')
#     plt.show()

In [74]:
# # Create a copy of the original image to draw the first line
# image_with_first_line = np.copy(bottom_image)

# # Extract the first line's rho and theta
# first_line = horizontal_lines[0]
# rho, theta = first_line

# # Calculate the endpoints of the line
# a = np.cos(theta)
# b = np.sin(theta)
# x0 = a * rho
# y0 = b * rho
# x1 = int(x0 + 1000 * (-b))
# y1 = int(y0 + 1000 * (a))
# x2 = int(x0 - 1000 * (-b))
# y2 = int(y0 - 1000 * (a))

# # Draw the first line on the image
# cv2.line(image_with_first_line, (x1, y1), (x2, y2), (0, 255, 0), 2)

# # Display the image
# plt.imshow(cv2.cvtColor(image_with_first_line, cv2.COLOR_BGR2RGB))
# plt.title("Image with First Detected Line")
# plt.axis('off')
# plt.show()

Try to automatize the detection

In [75]:
'''
Sructure of lines_p:
(x1, y1, x2, y2) are the coordinates of the two end points of the line segment
I want to convert them to the form rho,theta in order to easly detect the horizontal lines
'''
def convert_to_rho_theta(x1, y1, x2, y2):
    """
    Convert a line from (x1, y1, x2, y2) format to (rho, theta).
    """
    # Compute the line angle (in radians) and find the normal angle
    theta = np.arctan2(y2 - y1, x2 - x1) + np.pi / 2  # Normal angle

    # Compute rho using the normal equation
    rho = x1 * np.cos(theta) + y1 * np.sin(theta)

    # Ensure rho is positive by adjusting theta accordingly
    if rho < 0:
        rho = -rho
        theta += np.pi  # Shift by 180 degrees to maintain equivalence

    return rho, theta

In [76]:
# # Define the range for light brown color in HSV
# lower_brown = np.array([00, 30, 100])
# upper_brown = np.array([20, 200, 255])

# # Define the range for rose color in HSV
# lower_rose = np.array([150, 30, 200])
# upper_rose = np.array([180, 200, 255])

# # Tolleranza in radianti (5 gradi)
# tolerance = np.deg2rad(5)

# num_horizontal_lines = 0 # numero di linee orizzontali trovate
# num_frame = 0   # indice per scorrere i frames

# while num_horizontal_lines != 1: # finchè non trovo una sola linea orizzontale ---- PUO' NON ESSERE LA COSA OTTIMALE, INTANTO SEMBRA FUNZIONARE
#     # Set frame position
#     cap.set(cv2.CAP_PROP_POS_FRAMES, num_frame)
#     ret, image_color = cap.read()
#     if not ret:
#         print("Failed to read the frame")
#         cap.release()
#         exit()

#     # Convert the image to HSV color space
#     hsv_image = cv2.cvtColor(image_color, cv2.COLOR_BGR2HSV)

#     # Create masks for brown and rose colors
#     mask_brown = cv2.inRange(hsv_image, lower_brown, upper_brown)
#     mask_rose = cv2.inRange(hsv_image, lower_rose, upper_rose)

#     # Combine the masks
#     combined_mask = cv2.bitwise_or(mask_brown, mask_rose)

#     # apply brown and rose mask
#     extracted_image = cv2.bitwise_and(image_color, image_color, mask=combined_mask)

#     # Crop the bottom part of the image
#     limit_y = math.floor(3/4*extracted_image.shape[0])
#     bottom_image = extracted_image[limit_y:extracted_image.shape[0], 0:extracted_image.shape[1]]

#     # Convert the bottom image to grayscale
#     gray_image = cv2.cvtColor(bottom_image, cv2.COLOR_BGR2GRAY)

#     # Compute Otsu's threshold 
#     otsu_thresh, _ = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

#     # Set lower and upper thresholds relative to Otsu's threshold
#     lower = 0.5 * otsu_thresh
#     upper = 1.5 * otsu_thresh

#     # get edges
#     edges = cv2.Canny(gray_image, lower, upper)

#     # Apply Probabilistic Hough Line Transform (allow to set minLineLength and maxLineGap)
#     min_line_length = 50
#     max_line_gap = 10 
#     lines_p = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=min_line_length, maxLineGap=max_line_gap) 

#     # Convert all lines to (rho, theta) RICORDATI DI AGGIUNGERE LA FUNZIONE CHE STAI USANDO CHE POI NON TI FUNZIONA UN CAZZO E BESTEMMI
#     if lines_p is not None:
#         rho_theta_lines = []
#         for line in lines_p:
#             x1, y1, x2, y2 = line[0]  # Extract line coordinates
#             rho, theta = convert_to_rho_theta(x1, y1, x2, y2)
#             rho_theta_lines.append((rho, theta))

#     # Filtra le linee orizzontali
#     if rho_theta_lines is not None:
#         horizontal_lines = [line for line in rho_theta_lines if abs(line[1] - np.pi/2) <= tolerance]
#         print(f"Number of horizontal lines: {len(horizontal_lines)}")
  
#     num_horizontal_lines = len(horizontal_lines)
#     print('frame:', num_frame)
#     num_frame += 1

# # Create a copy of the original image to draw lines on
# line_image_horizontal = np.copy(image_color)

# # Convert the found line in the reference system of the original image
# adjusted_horizontal_lines = []
# for line in horizontal_lines:
#     rho, theta = line
#     # Adjust rho to account for the vertical shift
#     adjusted_rho = rho + limit_y * np.sin(theta)
#     adjusted_horizontal_lines.append((adjusted_rho, theta))

# # Draw the adjusted line on the original image
# for line in adjusted_horizontal_lines:
#     rho, theta = line
#     a = np.cos(theta)
#     b = np.sin(theta)
#     x0 = a * rho
#     y0 = b * rho
#     x1 = int(x0 + 1000 * (-b))
#     y1 = int(y0 + 1000 * (a))
#     x2 = int(x0 - 1000 * (-b))
#     y2 = int(y0 - 1000 * (a))
#     cv2.line(line_image_horizontal, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
# # Display the result
# plt.imshow(cv2.cvtColor(line_image_horizontal, cv2.COLOR_BGR2RGB))
# plt.title('Detected Lines with HoughLinesP')
# plt.axis('off')
# plt.show()


# '''
# Bisogna renderlo più robusto, perchè potrebbe rilevare sempre più linee orizzontali
# idea 1: prendo il frame con meno linee trovate e scelgo la prima 
# idea 2: dal primo frame mi salvo tutte le linee trovate e per quelli successivi tengo solo quelle che si trovano anche in quel frame, fino ad arrivare ad una sola linea
# '''

parte fuori dal while per vedere l'output se il ciclo non si ferma e lo fermo manualmente

In [77]:
# # Create a copy of the original image to draw lines on
# line_image_horizontal = np.copy(image_color)

# # Convert the found line in the reference system of the original image
# adjusted_horizontal_lines = []
# for line in horizontal_lines:
#     rho, theta = line
#     # Adjust rho to account for the vertical shift
#     adjusted_rho = rho + limit_y * np.sin(theta)
#     adjusted_horizontal_lines.append((adjusted_rho, theta))

# # Draw the adjusted line on the original image
# for line in adjusted_horizontal_lines:
#     rho, theta = line
#     a = np.cos(theta)
#     b = np.sin(theta)
#     x0 = a * rho
#     y0 = b * rho
#     x1 = int(x0 + 1000 * (-b))
#     y1 = int(y0 + 1000 * (a))
#     x2 = int(x0 - 1000 * (-b))
#     y2 = int(y0 - 1000 * (a))
#     cv2.line(line_image_horizontal, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
# # Display the result
# plt.imshow(cv2.cvtColor(line_image_horizontal, cv2.COLOR_BGR2RGB))
# plt.title('Detected Lines with HoughLinesP')
# plt.axis('off')
# plt.show()

In [78]:
# print(f"Rho: {adjusted_horizontal_lines[0][0]}, Theta: {adjusted_horizontal_lines[0][1]}")
# print(f"Rho: {horizontal_lines[0][0]}, theta: {horizontal_lines[0][1]}")

In [79]:
# # Define the path to save the CSV file
# intermediate_data_path = project_root / "notebook" / "lane_detection" / "intermediate_data" / "horizontal_line_2.csv"

# # Create a DataFrame with the horizontal line data
# horizontal_line_df = pd.DataFrame(adjusted_horizontal_lines, columns=['rho', 'theta'])

# # Save the DataFrame to a CSV file
# horizontal_line_df.to_csv(intermediate_data_path, index=False)

# print(f"Horizontal line saved to {intermediate_data_path}")

Prova su video

Funzione per la detection della linea

In [80]:
def bottom_detection(frame):

    # Define the range for light brown color in HSV
    lower_brown = np.array([00, 30, 100])
    upper_brown = np.array([20, 200, 255])

    # Define the range for rose color in HSV
    lower_rose = np.array([150, 30, 200])
    upper_rose = np.array([180, 200, 255])

    # Tolleranza in radianti (5 gradi)
    tolerance = np.deg2rad(5)
    
    # Convert the image to HSV color space
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create masks for brown and rose colors
    mask_brown = cv2.inRange(hsv_image, lower_brown, upper_brown)
    mask_rose = cv2.inRange(hsv_image, lower_rose, upper_rose)

    # Combine the masks
    combined_mask = cv2.bitwise_or(mask_brown, mask_rose)

    # apply brown and rose mask
    extracted_image = cv2.bitwise_and(frame, frame, mask=combined_mask)

    # Crop the bottom part of the image
    limit_y = math.floor(3/4*extracted_image.shape[0])
    bottom_image = extracted_image[limit_y:extracted_image.shape[0], 0:extracted_image.shape[1]]

    # Convert the bottom image to grayscale
    gray_image = cv2.cvtColor(bottom_image, cv2.COLOR_BGR2GRAY)

    # Compute Otsu's threshold 
    otsu_thresh, _ = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Set lower and upper thresholds relative to Otsu's threshold
    lower = 0.5 * otsu_thresh
    upper = 1.5 * otsu_thresh

    # get edges
    edges = cv2.Canny(gray_image, lower, upper)

    # Apply Probabilistic Hough Line Transform (allow to set minLineLength and maxLineGap)
    min_line_length = 50
    max_line_gap = 10 
    lines_p = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=min_line_length, maxLineGap=max_line_gap) 

    # Convert all lines to (rho, theta) 
    # RICORDATI DI AGGIUNGERE LA FUNZIONE CHE STAI USANDO CHE POI NON TI FUNZIONA UN CAZZO E BESTEMMI
    # FORSE NON è NECESSARIO TRASFORMARE IN RHO E THETA, ALLA FINE QUANDO FUNZIONA PULISCI IL CODICE
    if lines_p is not None:
        rho_theta_lines = []
        for line in lines_p:
            x1, y1, x2, y2 = line[0]  # Extract line coordinates
            rho, theta = convert_to_rho_theta(x1, y1, x2, y2)
            rho_theta_lines.append((rho, theta))

    # Filtra le linee orizzontali
    if rho_theta_lines is not None:
        horizontal_lines = [line for line in rho_theta_lines if abs(line[1] - np.pi/2) <= tolerance]
        print(f"Number of horizontal lines: {len(horizontal_lines)}")
        print('horizontal_lines:', horizontal_lines)

    if len(horizontal_lines) == 0:
        # if no horizontal lines are found, return an unfeasible value (vertical line at rho = 0)
        horizontal_line = [0, 0]
    else:
        # select the first horizontal line
        horizontal_line = horizontal_lines[0]

    # Convert the found line in the reference system of the original image
    adjusted_rho = horizontal_line[0] + limit_y * np.sin(horizontal_line[1])
    horizontal_line = (adjusted_rho, horizontal_line[1])

    # return the horizontal line
    return horizontal_line

Funzione per disegnare la linea sul frame

In [81]:
def write_line_on_frame(frame, line):
    # Create a copy of the original frame to draw the first line
    modified_frame = np.copy(frame)

    # Extract the first line's rho and theta
    if line:
        rho, theta = line

        # Calculate the endpoints of the line
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))

        # Draw the first line on the frame
        cv2.line(modified_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # return the modified frame
    return modified_frame

In [82]:
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = project_root / "data" / "recording_5" / "Recording_5.mp4" #/ "Recording_5_normal_speed.mp4"
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Reset the video to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Define the codec and create a VideoWriter object to save the modified frames
output_path = project_root / "data" / "recording_5" / "Bottom_line_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec for MP4 format
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))



# Initialize num_frame e horizontal_line
horizontal_lines = [None] * int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
num_frame = 0

# Loop through each frame in the video
while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or failed to read the frame:", num_frame)
        break

    # Perform operations on the current frame
    horizontal_lines[num_frame] = bottom_detection(frame)

    # Write the modified frame to the output video
    modified_frame = write_line_on_frame(frame, horizontal_lines[num_frame])

    # Write the modified frame to the output video
    out.write(modified_frame)

    # Increment the frame counter
    num_frame += 1

# Release the video capture and writer objects
# cap.release()
out.release()



Number of horizontal lines: 18
horizontal_lines: [(np.float64(209.4082473166071), np.float64(1.5869239606385328)), (np.float64(213.16171942255409), np.float64(1.5825604899446553)), (np.float64(194.12193772729773), np.float64(1.6001996149989017)), (np.float64(211.29583500362955), np.float64(1.5876486638362393)), (np.float64(204.00742718504353), np.float64(1.591845848931943)), (np.float64(211.5020434568296), np.float64(1.5834538786289485)), (np.float64(207.29780103102314), np.float64(1.587055056600026)), (np.float64(195.89909463408677), np.float64(1.5981867352697592)), (np.float64(215.07080503286394), np.float64(1.5877438566013016)), (np.float64(181.83546965990212), np.float64(1.605869657328122)), (np.float64(211.72512224198852), np.float64(1.5853584041876352)), (np.float64(214.86748373360658), np.float64(1.5825604899446553)), (np.float64(175.61011714523175), np.float64(1.6099919287219513)), (np.float64(246.00000000000003), np.float64(1.5707963267948966)), (np.float64(215.00000000000003)

In [83]:
# Convert horizontal_lines to a DataFrame
horizontal_lines_df = pd.DataFrame(horizontal_lines, columns=['rho', 'theta'])

# Save the DataFrame to a CSV file
output_data_path = project_root / "notebook" / "lane_detection" / "intermediate_data" / "horizontal_lines_5.csv"
horizontal_lines_df.to_csv(output_data_path, index=False)

print(f"Horizontal lines saved to {output_data_path}")

Horizontal lines saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\notebook\lane_detection\intermediate_data\horizontal_lines_5.csv
